# Adding selection
import some stuff!

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import ipywidgets as widgets
%matplotlib inline
%matplotlib widget

## The Moran model + constant selection
The difference between this and the original Moran model is that we now consider fitness in addition to frequency when drawing an individual to reproduce. This is represented by the `fit_ratio` parameter which described the ratio between species 1 and species 2 and is used to weight the frequencies of the species in the reproduction step. We use the `fit_ratio` to find the probability of selecting a member of species 1 for reproduction like this:
$$
P_1 = \frac{W_r f_1}{W_r f_1 + f_2}
$$
Where $W_r$ is the ratio of species 1 and species 2 absolute fitness and $f_i$ is the frequency of species $i$.

In [23]:
def moran_constant_selection(n_indiv, n_years, init_f1, fit_ratio):
    """ Implements the Moran model including constant selection and simulates it in time. 
        
        Parameters
        ----------
        n_indiv: int
            number of individuals in the community
        n_years: int
            number of potential 'turnovers' of the community
        init_f1: float
            initial frequency of species 1
        fit_ratio: float
            ratio of fitness of species 1 to fitness of species 2
            
        Return
        ------
        moran: np.array (n_years, 2)
            contains the species frequencies for each year of the simulation """

    # set up an empty array for the simulated frequencies
    # initialize with the given frequency. python counts from 0, sorrryyyy
    moran = np.zeros((n_years, 2))
    moran[0] = np.array([init_f1, 1 - init_f1])

    # get a vector representing the community. vellend calls this COM
    # this just makes a random vector drawn from the initial frequency
    comm = np.random.choice([0, 1], size=n_indiv, replace=True, p=moran[0])

    # now we can do the loop as in vellend. I'm going to write it a bit differently
    # but the idea is basically the same. (yes its slower)
    for year in range(1, n_years):

        # for each year we potentially replace each individual so we can loop
        # over that as well
        for indiv in range(n_indiv):
            # get probability of species 1
            f1 = np.sum(comm == 0) / n_indiv
            pr1 = fit_ratio * f1 / (fit_ratio * f1 + (1 - f1))

            # replace one individual with another
            death = np.random.randint(n_indiv)
            birth = np.random.choice([0, 1], p=[pr1, 1 - pr1])
            comm[death] = birth
        
        # when we're done looping over all of the individuals we can update
        # frequencies for the year (the timescale we care about)
        f1 = np.sum(comm == 0) / n_indiv
        moran[year] = np.array([f1, 1 - f1])

    return moran

## Running the simulations and visualization
As before we have a bunch of messy code to draw the plots. Once again we can change parameter values in the first block and not worry about the rest (unless you want to!)

In [24]:
def draw_simulation(iterations, individuals, years, initial_freq, fit_ratio):
    # the plot bit, this just makes a blank plot
    fig, ax = plt.subplots(ncols=2, figsize=(10,4), sharey=True, gridspec_kw = {'width_ratios':[3, 1]})
    # trajectory labels
    ax[0].set_xlabel('Years')
    ax[0].set_ylabel('Species 1 frequency')
    ax[0].set_ylim([0, 1])
    # distribution labels
    ax[1].set_xlabel('Count')
    ax[1].set_xlim([0, iterations])

    # we're going to plot a distribution too need an array for it
    dist = np.zeros(iterations)

    # we run the simulation and draw a trajectory
    for i in tqdm(range(iterations)):
        simulation = moran_constant_selection(individuals, years, initial_freq, fit_ratio)
        ax[0].plot(range(years), simulation[:, 0])
        # add final freq to our dist array
        dist[i] = simulation[-1, 0]

    # plot the distribution too
    ax[1].hist(dist, orientation='horizontal')
    ax[1].axhline(np.mean(dist), linestyle='--')


    plt.tight_layout()    
    plt.show()

# set up the interface
widgets.interact_manual(draw_simulation, iterations=(5, 50, 5), individuals=(10, 1000, 10), years=(5, 200, 5),
                  initial_freq=(0.0, 1.0, 0.01), fit_ratio=(0.5, 1.5, 0.01))

interactive(children=(IntSlider(value=25, description='iterations', max=50, min=5, step=5), IntSlider(value=50…

<function __main__.draw_simulation(iterations, individuals, years, initial_freq, fit_ratio)>